# Handwritten numbers Recognition

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras 

## Load Data

Extraer datos. Las imágenes son de 28*28 píxeles, por lo tanto el csv tiene 784 columnas más 1 indicando la salida

In [2]:
def extract():
    df_train = pd.read_csv('data/mnist_train.csv', delimiter = ',').to_numpy()
    df_test = pd.read_csv('data/mnist_test.csv', delimiter = ',').to_numpy()
    df = pd.DataFrame(np.concatenate((df_train,df_test)))
    return df

Transformar output a array de 10 y separar de las inputs

In [3]:
def transform(df): 
    y_raw = df.iloc[:, 0].to_numpy()
    x = df.drop(df.columns[0], axis='columns').to_numpy()
    y = []
    for i in range(0,y_raw.shape[0]):
        row=np.zeros(10)
        row[y_raw[i]]=1
        y.append(row)
    y=np.array(y)
    y.reshape(y.shape[0],10)
    return x, y

Creamos los dataset de entrenamiento y test

In [5]:
df = extract()
x_train,y_train = transform(df.iloc[0:60000])
#x_val,y_val = transform(df.iloc[50000:60000])
x_test,y_test = transform(df.iloc[60000:70000])

In [6]:
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(9998,28,28,1)

## Creación de la Red Neuronal

En la red neuronal vamoas a implementar una arquitectura de (784,512,256,128,10), con funciones de activación ReLu en todas las capas menos en la última, en la cual utilizaremos la funcion softmax (https://www.quora.com/Why-is-it-better-to-use-Softmax-function-than-sigmoid-function).<br> Para cada capa se utilizará la funcion Dense, la cual tiene los siguientes parámetros por defecto:
- activation=None,
- use_bias=True,
- kernel_initializer='glorot_uniform',
- bias_initializer='zeros',
- kernel_regularizer=None,
- bias_regularizer=None,
- activity_regularizer=None,
- kernel_constraint=None,
- bias_constraint=None




In [7]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
# next weight/high size = (n+2p-f)/s +1 
# n = current weight/high
# p = pad
# s = stride
# f = filter dim

Using TensorFlow backend.


## Sin pooling

Instanciamos el modelo, 2 capas convolucionales de 64 y 32

In [11]:
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

Entrenamiento, 3 epochs

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 94s 2ms/step - loss: 0.4955 - accuracy: 0.9309 - val_loss: 0.0682 - val_accuracy: 0.9782
Epoch 2/3
60000/60000 [==============================] - 94s 2ms/step - loss: 0.0543 - accuracy: 0.9832 - val_loss: 0.0645 - val_accuracy: 0.9785
Epoch 3/3
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0355 - accuracy: 0.9882 - val_loss: 0.0708 - val_accuracy: 0.9813


In [13]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 4s 354us/step
Test Acc:  0.9812962412834167


### Con Maxpooling

Añadimos una capa maxpool: 2*2, stride=1, padding = valid

In [8]:
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

Vamos a crear y entrenar el modelo, 3 epochs serán suficientes, el tiempo de entrenamiento se reduce un 50% aproximdamente.

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)#sin pool: 3min ish, acc:0.982096

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 47s 790us/step - loss: 0.5260 - accuracy: 0.9200 - val_loss: 0.0753 - val_accuracy: 0.9765
Epoch 2/3
60000/60000 [==============================] - 47s 776us/step - loss: 0.0642 - accuracy: 0.9803 - val_loss: 0.0714 - val_accuracy: 0.9770
Epoch 3/3
60000/60000 [==============================] - 45s 754us/step - loss: 0.0456 - accuracy: 0.9859 - val_loss: 0.0596 - val_accuracy: 0.9835


Evaluamos el modelo, los resultados mejoran.

In [10]:
tes_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 2s 204us/step
Test Acc:  0.9834967255592346


### Max Pooling 2

In [30]:
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

Esta vez con stride = 2

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 44s 728us/step - loss: 0.6904 - accuracy: 0.9187 - val_loss: 0.0835 - val_accuracy: 0.9741
Epoch 2/3
60000/60000 [==============================] - 44s 733us/step - loss: 0.0726 - accuracy: 0.9779 - val_loss: 0.0731 - val_accuracy: 0.9765
Epoch 3/3
60000/60000 [==============================] - 44s 737us/step - loss: 0.0501 - accuracy: 0.9845 - val_loss: 0.0631 - val_accuracy: 0.9807


Mejoran los timepos, baja un poco el rendimiento

In [16]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 2s 201us/step
Test Acc:  0.9806961417198181


### Common Structure

Usando padding=same

In [24]:
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))#capa de 64 filtros de 3x3. dims 64*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))#capa de 32 filtros de 3x3. dims 32*28*28*1
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))
model.add(Flatten())# pasa los resultados de matriz a vector
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))#dicho vector se conecta a una capa para generar la salida

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 44s 735us/step - loss: 0.6783 - accuracy: 0.8887 - val_loss: 0.1031 - val_accuracy: 0.9687
Epoch 2/3
60000/60000 [==============================] - 45s 757us/step - loss: 0.0831 - accuracy: 0.9749 - val_loss: 0.0764 - val_accuracy: 0.9764
Epoch 3/3
60000/60000 [==============================] - 46s 763us/step - loss: 0.0557 - accuracy: 0.9833 - val_loss: 0.0589 - val_accuracy: 0.9833


In [26]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 2s 216us/step
Test Acc:  0.9832966327667236


### Common Structure 2

In [31]:
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))
#Compile and train
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size=200, epochs=3)
#Testing
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

Train on 60000 samples, validate on 9998 samples
Epoch 1/3
60000/60000 [==============================] - 46s 764us/step - loss: 0.6860 - accuracy: 0.8882 - val_loss: 0.1396 - val_accuracy: 0.9603
Epoch 2/3
60000/60000 [==============================] - 47s 778us/step - loss: 0.1014 - accuracy: 0.9705 - val_loss: 0.1159 - val_accuracy: 0.9666
Epoch 3/3
9998/9998 [==============================] - 2s 210us/step
Test Acc:  0.9777955412864685
